<a href="https://colab.research.google.com/github/ShrutiChrist/Natural-Language-Processing/blob/main/2348545_lab9(NLP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Edit Distance

In [4]:
def editDistance(str1, str2, m, n):
    if m == 0:
        return n
    if n == 0:
        return m

    # If last characters of two strings are same, nothing
    # much to do. Ignore last characters and get count for
    # remaining strings.
    if str1[m-1] == str2[n-1]:
        return editDistance(str1, str2, m-1, n-1)

    # If last characters are not same, consider all three
    # operations on last character of first string, recursively
    # compute minimum cost for all three operations and take
    # minimum of three values.
    return 1 + min(editDistance(str1, str2, m, n-1),    # Inser22
                   editDistance(str1, str2, m-1, n),    # Remove
                   editDistance(str1, str2, m-1, n-1)    # Replace
                   )


In [5]:
str1=input("Enter First String:")
str2=input("Enter Second String:")
print(editDistance(str1, str2, len(str1), len(str2)))

Enter First String:shruti
Enter Second String:shru
2


## Poem Generation

### Importing Libraries

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras.utils as ku
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

### Reading the data

In [7]:
data = open('/content/poem.txt', encoding="utf8").read()
corpus = data.lower().split("\n")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index)

print("Total Words:", total_words)

Total Words: 197


### Text to Embedding

In [8]:
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
predictors, label = input_sequences[:, :-1], input_sequences[:, -1]
label = ku.to_categorical(label, num_classes=total_words+1)

### Bi-directional LSTM Model

In [9]:
model = Sequential()
model.add(Embedding(total_words+1, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words+1/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words+1, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 11, 100)           19800     
                                                                 
 bidirectional (Bidirection  (None, 11, 300)           301200    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 11, 300)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               160400    
                                                                 
 dense (Dense)               (None, 197)               19897     
                                                                 
 dense_1 (Dense)             (None, 198)               39204     
                                                        

In [10]:
history = model.fit(predictors, label, epochs=100, verbose=1)

Epoch 1/100
11/11 [==============================] - 8s 59ms/step - loss: 6.4690 - accuracy: 0.0685
Epoch 2/100
11/11 [==============================] - 1s 60ms/step - loss: 5.9837 - accuracy: 0.0804
Epoch 3/100
11/11 [==============================] - 1s 59ms/step - loss: 5.6465 - accuracy: 0.0893
Epoch 4/100
11/11 [==============================] - 1s 61ms/step - loss: 5.4750 - accuracy: 0.0804
Epoch 5/100
11/11 [==============================] - 1s 59ms/step - loss: 5.3514 - accuracy: 0.0744
Epoch 6/100
11/11 [==============================] - 1s 59ms/step - loss: 5.2254 - accuracy: 0.0714
Epoch 7/100
11/11 [==============================] - 1s 99ms/step - loss: 5.0980 - accuracy: 0.0833
Epoch 8/100
11/11 [==============================] - 1s 115ms/step - loss: 4.9802 - accuracy: 0.0744
Epoch 9/100
11/11 [==============================] - 1s 78ms/step - loss: 4.8497 - accuracy: 0.0893
Epoch 10/100
11/11 [==============================] - 1s 64ms/step - loss: 4.6982 - accuracy: 0.077

### Poem Generation

In [11]:
seed_text = input("Enter first few words")
next_words = 50
ouptut_text = ""

for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences(
        [token_list], maxlen=max_sequence_len-1,
      padding='pre')
    predicted = np.argmax(model.predict(token_list,
                                        verbose=0), axis=-1)
    output_word = ""

    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break

    seed_text += " " + output_word

print(seed_text)

Enter first few wordsnature
nature in the heart of a forest where shadows dance with light light light light day day life strife light light light with light light light with gold with light light light with gold with gold with with gold with gold with with gold with with gold with with gold with
